#### **Task 3**: Does the weather affect the delay? 
Use the API to pull the weather information for flights. There is no need to get weather for ALL flights. We can choose the right representative sample. Let's focus on four weather types:
- sunny
- cloudy
- rainy
- snow.

Test the hypothesis that these 4 delays are from the same distribution. If they are not, which ones are significantly different?

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
%matplotlib inline
import scipy.stats as st

### Import Weather Station Data

In [44]:
 # metadata for weather station locations
    
df_weather_stn = pd.read_csv("usa-ghcnd-stations.csv", sep=",")

In [45]:
df_weather_stn

,station_id,latitude,longitude,elevation,state,name
0,US009052008,43.7333,-96.6333,482.0,SD,SIOUX FALLS (ENVIRON. CANADA)
1,US10RMHS145,40.5268,-105.1113,1569.1,CO,RMHS 1.6 SSW
2,US10adam001,40.5680,-98.5069,598.0,NE,JUNIATA 1.5 S
3,US10adam002,40.5093,-98.5493,601.1,NE,JUNIATA 6.0 SSW
4,US10adam003,40.4663,-98.6537,615.1,NE,HOLSTEIN 0.1 NW
...,...,...,...,...,...,...
69601,USW00096405,60.4731,-145.3542,25.3,AK,CORDOVA 14 ESE CRN
69602,USW00096406,64.5014,-154.1297,78.9,AK,RUBY 44 ESE CRN 70224
69603,USW00096407,66.5620,-159.0036,6.7,AK,SELAWIK 28 E CRN 70170
69604,USW00096408,63.4519,-150.8747,678.2,AK,DENALI 27 N CRN 70244


## Find Weather Station closest to Each Airport

1. Find all airports from flights.csv
    - get 20k flights where arrival delay is > 0
2. For each airport, get lat and long
3. Get closest weather station to each airport
4. Get historical weather data based on weather station and flight date

In [46]:
# import CSV file arrival delay outlier data cleaned
df_flights = pd.read_csv('flights_arr_delay_for_weather.csv')
df_flights.head()

,fl_date,mkt_unique_carrier,op_unique_carrier,op_carrier_fl_num,origin,origin_city_name,dest,dest_city_name,crs_dep_time,dep_time,...,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2019-04-20,AA,AA,2244,ORD,"Chicago, IL",DFW,"Dallas/Fort Worth, TX",1311,1303.0,...,-35.0,153,126.0,101.0,801,0.0,0.0,0.0,0.0,0.0
1,2019-02-13,WN,WN,1731,MSY,"New Orleans, LA",HOU,"Houston, TX",2130,2159.0,...,18.0,75,64.0,55.0,302,0.0,0.0,0.0,0.0,18.0
2,2018-11-20,AA,AA,2620,DFW,"Dallas/Fort Worth, TX",IND,"Indianapolis, IN",1834,1833.0,...,13.0,128,142.0,102.0,761,0.0,0.0,0.0,0.0,0.0
3,2018-08-28,DL,9E,4060,JFK,"New York, NY",IAD,"Washington, DC",1359,1353.0,...,-34.0,98,70.0,51.0,228,0.0,0.0,0.0,0.0,0.0
4,2018-10-30,DL,OO,3807,SLC,"Salt Lake City, UT",SMF,"Sacramento, CA",1535,1533.0,...,-7.0,112,107.0,81.0,532,0.0,0.0,0.0,0.0,0.0


In [47]:
df_flights.shape

(177910, 25)

In [48]:
df_flights.drop(columns=['distance', 
                        'op_carrier_fl_num', 
                        'origin_city_name', 
                        'crs_dep_time', 
                        'dep_time', 
                        'crs_arr_time', 
                        'crs_elapsed_time', 
                        'actual_elapsed_time', 
                        'dest_city_name', 
                        'taxi_out', 
                        'taxi_in', 
                        'arr_time'],
                inplace=True
               )

Get flights where arrival delay is greater than 0

In [49]:
df_flights_arr_delay = df_flights.loc[(df_flights['arr_delay'] > 0)]
df_flights_arr_delay.count()

fl_date                50695
mkt_unique_carrier     50695
op_unique_carrier      50695
origin                 50695
dest                   50695
dep_delay              50695
arr_delay              50695
air_time               50695
carrier_delay          50695
weather_delay          50695
nas_delay              50695
security_delay         50695
late_aircraft_delay    50695
dtype: int64

In [50]:
df_flights_arr_delay_20k = df_flights_arr_delay.sample(n=20000) # get random 20K

In [51]:
df_flights_arr_delay_20k.shape

(20000, 13)

# Get Airports from the Flight Data

In [52]:
# import airport data

df_airports = pd.read_csv('../airport/airports_usa_info_data.csv')
df_airports.head()

,type,name,iso_country,iso_region,municipality,iata_code,Longitude,Latitude
0,medium_airport,Aleknagik / New Airport,US,US-AK,Aleknagik,WKK,-158.617996,59.282600
1,medium_airport,South Alabama Regional At Bill Benton Field Ai...,US,US-AL,Andalusia/Opp,NaN,-86.393799,31.308800
2,medium_airport,Lehigh Valley International Airport,US,US-PA,Allentown,ABE,-75.440804,40.652100
3,medium_airport,Abilene Regional Airport,US,US-TX,Abilene,ABI,-99.681900,32.411301
4,large_airport,Albuquerque International Sunport,US,US-NM,Albuquerque,ABQ,-106.609001,35.040199


In [53]:
df_airports.rename(columns={"Longitude" : "aport_long", 
                            "Latitude" : "aport_lat", 
                            "iata_code" : "aport_iata", 
                            "name" : "aport_name", 
                            "iso_country" : "aport_country", 
                            "municipality" : "aport_muni",
                            "iso_region" : "aport_region"},
                   inplace=True
                  )

df_airports.drop(columns=["type"], inplace=True)

In [54]:
df_airports.head()

,aport_name,aport_country,aport_region,aport_muni,aport_iata,aport_long,aport_lat
0,Aleknagik / New Airport,US,US-AK,Aleknagik,WKK,-158.617996,59.282600
1,South Alabama Regional At Bill Benton Field Ai...,US,US-AL,Andalusia/Opp,NaN,-86.393799,31.308800
2,Lehigh Valley International Airport,US,US-PA,Allentown,ABE,-75.440804,40.652100
3,Abilene Regional Airport,US,US-TX,Abilene,ABI,-99.681900,32.411301
4,Albuquerque International Sunport,US,US-NM,Albuquerque,ABQ,-106.609001,35.040199


In [55]:
df_flights_arr_delay_20k = df_flights_arr_delay_20k.merge(df_airports, left_on='dest', right_on='aport_iata')

In [56]:
## Using this stackoverflow answer to find closest https://stackoverflow.com/questions/41336756/find-the-closest-latitude-and-longitude
from math import cos, asin, sqrt
from scipy.spatial.distance import cdist

def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

def match_value(df, col1, x, col2):
    """ Match value x from col1 row to value in col2. """
    return df[df[col1] == x][col2].values[0]

df1 = df_weather_stn.copy()
df2 = df_flights_arr_delay_20k.copy()

df1['aport_coord'] = [(x, y) for x,y in zip(df1['latitude'], df1['longitude'])]
df2['aport_coord'] = [(x, y) for x,y in zip(df2['aport_lat'], df2['aport_long'])]

df2['closest_weather_stn'] = [closest_point(x, list(df1['aport_coord'])) for x in df2['aport_coord']]
df2['weather_stn_id'] = [match_value(df1, 'aport_coord', x, 'station_id') for x in df2['closest_weather_stn']]

df_flights_arr_delay_20k = df2.copy()

In [57]:
df_flights_arr_delay_20k.shape

(19830, 23)

In [58]:
df_flights_arr_delay_20k.head()

,fl_date,mkt_unique_carrier,op_unique_carrier,origin,dest,dep_delay,arr_delay,air_time,carrier_delay,weather_delay,...,aport_name,aport_country,aport_region,aport_muni,aport_iata,aport_long,aport_lat,aport_coord,closest_weather_stn,weather_stn_id
0,2018-05-22,WN,WN,HOU,MAF,22.0,12.0,63.0,0.0,0.0,...,Midland International Airport,US,US-TX,Midland,MAF,-102.202003,31.942499,"(31.9424991607666, -102.2020034790039)","(31.9425, -102.1894)",USC00415893
1,2018-06-29,AA,YV,DFW,MAF,1.0,2.0,47.0,0.0,0.0,...,Midland International Airport,US,US-TX,Midland,MAF,-102.202003,31.942499,"(31.9424991607666, -102.2020034790039)","(31.9425, -102.1894)",USC00415893
2,2019-09-17,AA,YV,PHX,MAF,-1.0,5.0,78.0,0.0,0.0,...,Midland International Airport,US,US-TX,Midland,MAF,-102.202003,31.942499,"(31.9424991607666, -102.2020034790039)","(31.9425, -102.1894)",USC00415893
3,2018-02-23,WN,WN,DAL,MAF,-1.0,1.0,59.0,0.0,0.0,...,Midland International Airport,US,US-TX,Midland,MAF,-102.202003,31.942499,"(31.9424991607666, -102.2020034790039)","(31.9425, -102.1894)",USC00415893
4,2019-12-27,UA,UA,IAH,MAF,14.0,4.0,65.0,0.0,0.0,...,Midland International Airport,US,US-TX,Midland,MAF,-102.202003,31.942499,"(31.9424991607666, -102.2020034790039)","(31.9425, -102.1894)",USC00415893


## Import Weather Data by Year

In [59]:
colnames = ['weather_stn_id', 'weather_date', 'weather_element', 'weather_element_val', "weather_MFLAG1", "weather_QFLAG1", "weather_VALUE2", "weather_MFLAG2"]

weather data documentation: https://www.ncei.noaa.gov/pub/data/ghcn/daily/readme.txt

### Import weather data 2018, 2019, 2020

In [64]:
weather_data_2018 = pd.read_csv('2018.csv', names=colnames, header=None)
weather_data_2019 = pd.read_csv('2019.csv', names=colnames, header=None)
weather_data_2020 = pd.read_csv('2020.csv', names=colnames, header=None)

In [65]:
# df_weather_data = pd.concat([weather_data_2018, weather_data_2019, weather_data_2020])

### 2018 weather data in USA

In [66]:
# filter station data by weather station_id in the USA only, using the first 2 letters of the weather_stn_id

us_weather_data_2018 = weather_data_2018[weather_data_2018['weather_stn_id'].str[:2].eq('US')]

In [67]:
us_weather_data_2018

,weather_stn_id,weather_date,weather_element,weather_element_val,weather_MFLAG1,weather_QFLAG1,weather_VALUE2,weather_MFLAG2
29067,US10adam002,20180101,PRCP,0,NaN,NaN,N,700.0
29068,US10adam004,20180101,PRCP,0,T,NaN,N,700.0
29069,US10adam008,20180101,PRCP,0,NaN,NaN,N,700.0
29070,US10adam022,20180101,PRCP,0,NaN,NaN,N,700.0
29071,US10adam032,20180101,PRCP,0,NaN,NaN,N,700.0
...,...,...,...,...,...,...,...,...
36333036,USW00096408,20181231,TMAX,27,NaN,NaN,R,NaN
36333037,USW00096408,20181231,TMIN,-63,NaN,NaN,R,NaN
36333038,USW00096408,20181231,PRCP,0,NaN,NaN,R,NaN
36333039,USW00096409,20181231,TMAX,-21,NaN,NaN,R,NaN


In [68]:
# export 2018 weather data to csv
us_weather_data_2018.to_csv('us_weather_data_2018.csv', index=False)

### 2019 weather data in USA

In [69]:
us_weather_data_2019 = weather_data_2019[weather_data_2019['weather_stn_id'].str[:2].eq('US')]

In [70]:
us_weather_data_2019

,weather_stn_id,weather_date,weather_element,weather_element_val,weather_MFLAG1,weather_QFLAG1,weather_VALUE2,weather_MFLAG2
28335,US10adam002,20190101,PRCP,0,T,NaN,N,700.0
28336,US10adam004,20190101,PRCP,0,NaN,NaN,N,700.0
28337,US10adam008,20190101,PRCP,0,NaN,NaN,N,700.0
28338,US10adam010,20190101,PRCP,0,NaN,NaN,N,700.0
28339,US10adam022,20190101,PRCP,0,NaN,NaN,N,700.0
...,...,...,...,...,...,...,...,...
35949188,USW00096408,20191231,TMIN,-218,NaN,NaN,R,NaN
35949189,USW00096408,20191231,PRCP,14,NaN,NaN,R,NaN
35949190,USW00096409,20191231,TMAX,-294,NaN,NaN,R,NaN
35949191,USW00096409,20191231,TMIN,-311,NaN,NaN,R,NaN


In [71]:
# export 2019 weather data to csv
us_weather_data_2019.to_csv('us_weather_data_2019.csv', index=False)

### 2020 weather data in USA

In [72]:
us_weather_data_2020 = weather_data_2020[weather_data_2020['weather_stn_id'].str[:2].eq('US')]

In [73]:
us_weather_data_2020

,weather_stn_id,weather_date,weather_element,weather_element_val,weather_MFLAG1,weather_QFLAG1,weather_VALUE2,weather_MFLAG2
27296,US10adam002,20200101,PRCP,0,NaN,NaN,N,700.0
27297,US10adam004,20200101,PRCP,0,NaN,NaN,N,700.0
27298,US10adam008,20200101,PRCP,0,NaN,NaN,N,700.0
27299,US10adam010,20200101,PRCP,0,NaN,NaN,N,700.0
27300,US10adam022,20200101,PRCP,0,NaN,NaN,N,700.0
...,...,...,...,...,...,...,...,...
36287420,USW00096408,20201231,TMIN,-115,NaN,NaN,R,NaN
36287421,USW00096408,20201231,PRCP,0,NaN,NaN,R,NaN
36287422,USW00096409,20201231,TMAX,-189,NaN,NaN,R,NaN
36287423,USW00096409,20201231,TMIN,-257,NaN,NaN,R,NaN


In [97]:
# export 2020 weather data to csv
us_weather_data_2020.to_csv('us_weather_data_2020.csv', index=False)

In [98]:
all_us_weather_data = pd.concat([us_weather_data_2018, us_weather_data_2019, us_weather_data_2020])

In [99]:
all_us_weather_data.head()

,weather_stn_id,weather_date,weather_element,weather_element_val,weather_MFLAG1,weather_QFLAG1,weather_VALUE2,weather_MFLAG2
29067,US10adam002,20180101,PRCP,0,NaN,NaN,N,700.0
29068,US10adam004,20180101,PRCP,0,T,NaN,N,700.0
29069,US10adam008,20180101,PRCP,0,NaN,NaN,N,700.0
29070,US10adam022,20180101,PRCP,0,NaN,NaN,N,700.0
29071,US10adam032,20180101,PRCP,0,NaN,NaN,N,700.0


In [100]:
all_us_weather_data.dtypes

weather_stn_id          object
weather_date             int64
weather_element         object
weather_element_val      int64
weather_MFLAG1          object
weather_QFLAG1          object
weather_VALUE2          object
weather_MFLAG2         float64
dtype: object

## Find Weather on Flight Date

In [101]:
# df_flights_arr_delay_20k['fl_date'].astype(datetime)

In [102]:
# df_delay = df_flights_arr_delay_20k[['fl_date', 'arr_delay', 'weather_stn_id']]
# df_merge = pd.merge(df_delay, df_weather, how = 'left', right_on=['weather_stn_id', 'weather_date'], left_on=['weather_stn_id', 'fl_date'])

In [103]:
# df_weather = all_us_weather_data[['weather_stn_id', 'weather_date', 'weather_element']]
# df_weather.

In [104]:
# join
# find what weather was for this date
# create additional column 
# weather element and delay
# calculate correlation grouby, avg delay 

In [105]:
df_flights_arr_delay_20k.tail(5)

,fl_date,mkt_unique_carrier,op_unique_carrier,origin,dest,dep_delay,arr_delay,air_time,carrier_delay,weather_delay,...,aport_country,aport_region,aport_muni,aport_iata,aport_long,aport_lat,aport_coord,closest_weather_stn,weather_stn_id,dest_weather
19825,2018-07-03,AS,AS,ANC,OTZ,-4.0,2.0,78.0,0.0,0.0,...,US,US-AK,Kotzebue,OTZ,-162.598999,66.884697,"(66.88469696, -162.598999)","(66.8667, -162.6333)",USW00026616,NaN
19826,2019-12-10,DL,OO,MSP,XWA,24.0,3.0,93.0,0.0,0.0,...,US,US-ND,Williston,XWA,-103.748797,48.258387,"(48.258387, -103.748797)","(48.2542, -103.7467)",USC00329426,NaN
19827,2019-11-11,DL,OO,ATL,BQK,30.0,23.0,41.0,0.0,0.0,...,US,US-GA,Brunswick,BQK,-81.466499,31.258801,"(31.2588005065918, -81.46649932861328)","(31.25, -81.4667)",USW00093836,NaN
19828,2018-12-24,AS,AS,SCC,BRW,34.0,29.0,37.0,0.0,0.0,...,US,US-AK,UtqiaÃ„Â¡vik,BRW,-156.766008,71.285402,"(71.285402, -156.766008)","(71.2833, -156.7814)",USW00027502,NaN
19829,2019-02-05,DL,OO,MSP,BJI,-5.0,16.0,34.0,0.0,0.0,...,US,US-MN,Bemidji,BJI,-94.933701,47.509399,"(47.50939941, -94.93370056)","(47.5033, -94.9281)",USR0000MBEM,NaN


In [106]:
df_flights_arr_delay_20k['fl_date'] = pd.to_datetime(df_flights_arr_delay_20k['fl_date'], format='%Y-%m-%d', errors='ignore')

In [107]:
all_us_weather_data['weather_date'] = pd.to_datetime(all_us_weather_data['weather_date'], format='%Y%m%d', errors='ignore')

In [108]:
all_us_weather_data.head()

,weather_stn_id,weather_date,weather_element,weather_element_val,weather_MFLAG1,weather_QFLAG1,weather_VALUE2,weather_MFLAG2
29067,US10adam002,2018-01-01,PRCP,0,NaN,NaN,N,700.0
29068,US10adam004,2018-01-01,PRCP,0,T,NaN,N,700.0
29069,US10adam008,2018-01-01,PRCP,0,NaN,NaN,N,700.0
29070,US10adam022,2018-01-01,PRCP,0,NaN,NaN,N,700.0
29071,US10adam032,2018-01-01,PRCP,0,NaN,NaN,N,700.0


In [109]:
# created placeholder holder column for destination airport weather on flight date
df_flights_arr_delay_20k["dest_weather"] = np.nan

In [110]:
df_flights_arr_delay_20k.head(1)

,fl_date,mkt_unique_carrier,op_unique_carrier,origin,dest,dep_delay,arr_delay,air_time,carrier_delay,weather_delay,...,aport_country,aport_region,aport_muni,aport_iata,aport_long,aport_lat,aport_coord,closest_weather_stn,weather_stn_id,dest_weather
0,2018-05-22,WN,WN,HOU,MAF,22.0,12.0,63.0,0.0,0.0,...,US,US-TX,Midland,MAF,-102.202003,31.942499,"(31.9424991607666, -102.2020034790039)","(31.9425, -102.1894)",USC00415893,NaN


In [111]:
df_flights_weather_delay = df_flights.loc[(df_flights['weather_delay'] > 0)]
df_flights_weather_delay.count()

fl_date                662
mkt_unique_carrier     662
op_unique_carrier      662
origin                 662
dest                   662
dep_delay              662
arr_delay              662
air_time               662
carrier_delay          662
weather_delay          662
nas_delay              662
security_delay         662
late_aircraft_delay    662
dtype: int64

In [112]:
df_flights_weather_delay = df_flights_weather_delay.merge(df_airports, left_on='dest', right_on='aport_iata')
df_flights_weather_delay.rename(columns={"aport_name" : "dest_aport_name",
                                 "aport_region" : "dest_aport_region",
                                 "aport_country" : "dest_aport_country",
                                 "aport_muni" : "dest_aport_muni",
                                 "aport_iata" : "dest_aport_iata",
                                 "aport_iata" : "dest_aport_iata",
                                 "aport_long" : "dest_aport_long",
                                 "aport_lat" : "dest_aport_lat"
                                }, inplace = True)

In [113]:
df_flights_weather_delay = df_flights_weather_delay.merge(df_airports, left_on='origin', right_on='aport_iata')
df_flights_weather_delay.rename(columns={"aport_name" : "origin_aport_name",
                                 "aport_region" : "origin_aport_region",
                                 "aport_country" : "origin_aport_country",
                                 "aport_muni" : "origin_aport_muni",
                                 "aport_iata" : "origin_aport_iata",
                                 "aport_iata" : "origin_aport_iata",
                                 "aport_long" : "origin_aport_long",
                                 "aport_lat" : "origin_aport_lat"
                                }, inplace = True)

In [114]:
df_flights_weather_delay.head(2)

,fl_date,mkt_unique_carrier,op_unique_carrier,origin,dest,dep_delay,arr_delay,air_time,carrier_delay,weather_delay,...,dest_aport_iata,dest_aport_long,dest_aport_lat,origin_aport_name,origin_aport_country,origin_aport_region,origin_aport_muni,origin_aport_iata,origin_aport_long,origin_aport_lat
0,2018-10-17,AA,MQ,DFW,TYR,35.0,40.0,27.0,0.0,35.0,...,TYR,-95.402397,32.354099,Dallas Fort Worth International Airport,US,US-TX,Dallas-Fort Worth,DFW,-97.038002,32.896801
1,2019-02-02,AA,MQ,DFW,TYR,25.0,34.0,22.0,0.0,25.0,...,TYR,-95.402397,32.354099,Dallas Fort Worth International Airport,US,US-TX,Dallas-Fort Worth,DFW,-97.038002,32.896801


In [115]:
df_flights_weather_delay['fl_date'] = pd.to_datetime(df_flights_weather_delay['fl_date'], format='%Y-%m-%d', errors='ignore')
df_flights_weather_delay["origin_weather"] = np.nan
df_flights_weather_delay["dest_weather"] = np.nan

In [116]:
## Using this stackoverflow answer to find closest https://stackoverflow.com/questions/41336756/find-the-closest-latitude-and-longitude
from math import cos, asin, sqrt
from scipy.spatial.distance import cdist

def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

def match_value(df, col1, x, col2):
    """ Match value x from col1 row to value in col2. """
    return df[df[col1] == x][col2].values[0]

df1 = df_weather_stn.copy()
df2 = df_flights_weather_delay.copy()

df1['aport_coord'] = [(x, y) for x,y in zip(df1['latitude'], df1['longitude'])]
df2['origin_aport_coord'] = [(x, y) for x,y in zip(df2['origin_aport_lat'], df2['origin_aport_long'])]
df2['dest_aport_coord'] = [(x, y) for x,y in zip(df2['dest_aport_lat'], df2['dest_aport_long'])]

df2['origin_closest_weather_stn'] = [closest_point(x, list(df1['aport_coord'])) for x in df2['origin_aport_coord']]
df2['dest_closest_weather_stn'] = [closest_point(x, list(df1['aport_coord'])) for x in df2['dest_aport_coord']]
df2['origin_weather_stn_id'] = [match_value(df1, 'aport_coord', x, 'station_id') for x in df2['origin_closest_weather_stn']]
df2['dest_weather_stn_id'] = [match_value(df1, 'aport_coord', x, 'station_id') for x in df2['dest_closest_weather_stn']]

df_flights_weather_delay = df2.copy()

In [117]:
df_flights_weather_delay.head()

,fl_date,mkt_unique_carrier,op_unique_carrier,origin,dest,dep_delay,arr_delay,air_time,carrier_delay,weather_delay,...,origin_aport_long,origin_aport_lat,origin_weather,dest_weather,origin_aport_coord,dest_aport_coord,origin_closest_weather_stn,dest_closest_weather_stn,origin_weather_stn_id,dest_weather_stn_id
0,2018-10-17,AA,MQ,DFW,TYR,35.0,40.0,27.0,0.0,35.0,...,-97.038002,32.896801,NaN,NaN,"(32.896801, -97.038002)","(32.35409927368164, -95.40239715576172)","(32.8978, -97.0189)","(32.3542, -95.4025)",USW00003927,USW00013972
1,2019-02-02,AA,MQ,DFW,TYR,25.0,34.0,22.0,0.0,25.0,...,-97.038002,32.896801,NaN,NaN,"(32.896801, -97.038002)","(32.35409927368164, -95.40239715576172)","(32.8978, -97.0189)","(32.3542, -95.4025)",USW00003927,USW00013972
2,2019-09-20,AA,AA,DFW,BOS,29.0,30.0,204.0,0.0,9.0,...,-97.038002,32.896801,NaN,NaN,"(32.896801, -97.038002)","(42.36429977, -71.00520325)","(32.8978, -97.0189)","(42.3606, -71.0106)",USW00003927,USW00014739
3,2019-06-09,AA,MQ,DFW,LIT,26.0,17.0,45.0,0.0,17.0,...,-97.038002,32.896801,NaN,NaN,"(32.896801, -97.038002)","(34.729400634799994, -92.2242965698)","(32.8978, -97.0189)","(34.7272, -92.2389)",USW00003927,USW00013963
4,2018-10-07,DL,DL,DFW,MSP,54.0,35.0,108.0,0.0,14.0,...,-97.038002,32.896801,NaN,NaN,"(32.896801, -97.038002)","(44.882, -93.221802)","(32.8978, -97.0189)","(44.8833, -93.2167)",USW00003927,USW00014947


In [ ]:
# iterate through all the flights
# get the weather station data closest to the destination aiport on the flight date
# classify 

for index, row in df_flights_weather_delay.iterrows():
    flight_date = row['fl_date']
    origin_weather_stn_id = row['origin_weather_stn_id']
    dest_weather_stn_id = row['dest_weather_stn_id']
    flight_weather_delay = row['weather_delay']
    flight_arr_delay = row['arr_delay']
    
    # get weather data for the destination airport on the flight date
    df_origin_weather_stn = all_us_weather_data[((all_us_weather_data['weather_stn_id'] == origin_weather_stn_id) & (all_us_weather_data['weather_date'] == flight_date))]
    df_dest_weather_stn = all_us_weather_data[((all_us_weather_data['weather_stn_id'] == dest_weather_stn_id) & (all_us_weather_data['weather_date'] == flight_date))]
    
    # get origin weather
    precip_ele = df_origin_weather_stn.index[df_origin_weather_stn['weather_element'] == 'PRCP'].tolist()
    snow_ele = df_origin_weather_stn.index[df_origin_weather_stn['weather_element'] == 'SNOW'].tolist()
    fog_ele_1 = df_origin_weather_stn.index[df_origin_weather_stn['weather_element'] == 'WT01'].tolist()
    fog_ele_2 = df_origin_weather_stn.index[df_origin_weather_stn['weather_element'] == 'WT02'].tolist()
    thunder_ele = df_origin_weather_stn.index[df_origin_weather_stn['weather_element'] == 'WT03'].tolist()
    
    origin_weather = "Clear" 
    
    if precip_ele:
        precip_val = df_origin_weather_stn['weather_element_val'][precip_ele[0]]
        if precip_val > 0:
            origin_weather = "Rainy"

    if snow_ele:
        snow_val = df_origin_weather_stn['weather_element_val'][snow_ele[0]]
        if snow_val > 0:
            origin_weather = "Snowy"

    if fog_ele_1:
        fog_val = df_origin_weather_stn['weather_element_val'][fog_ele_1[0]]
        if fog_val > 0:
            origin_weather = "Foggy"
            
    if thunder_ele:
        thunder_ele = df_origin_weather_stn['weather_element_val'][thunder_ele[0]]
        if thunder_ele > 0:
            origin_weather = "Thunder"
    
    # get destination weather
    precip_ele = df_dest_weather_stn.index[df_dest_weather_stn['weather_element'] == 'PRCP'].tolist()
    snow_ele = df_dest_weather_stn.index[df_dest_weather_stn['weather_element'] == 'SNOW'].tolist()
    fog_ele_1 = df_dest_weather_stn.index[df_dest_weather_stn['weather_element'] == 'WT01'].tolist()
    fog_ele_2 = df_dest_weather_stn.index[df_dest_weather_stn['weather_element'] == 'WT02'].tolist()
    thunder_ele = df_dest_weather_stn.index[df_dest_weather_stn['weather_element'] == 'WT03'].tolist()
    
    dest_weather = "Clear" 
    
    if precip_ele:
        precip_val = df_dest_weather_stn['weather_element_val'][precip_ele[0]]
        if precip_val > 0:
            dest_weather = "Rainy"

    if snow_ele:
        snow_val = df_dest_weather_stn['weather_element_val'][snow_ele[0]]
        if snow_val > 0:
            dest_weather = "Snowy"

    if fog_ele_1:
        fog_val = df_dest_weather_stn['weather_element_val'][fog_ele_1[0]]
        if fog_val > 0:
            dest_weather = "Foggy"
            
    if thunder_ele:
        thunder_ele = df_dest_weather_stn['weather_element_val'][thunder_ele[0]]
        if thunder_ele > 0:
            dest_weather = "Thunder"
            
    df_flights_weather_delay.at[index,'origin_weather'] = origin_weather
    df_flights_weather_delay.at[index,'dest_weather'] = dest_weather
    # df_flights_weather_delay.loc[index]

    # print(f"Arrival Delay: {flight_arr_delay}, Weather delay: {flight_weather_delay}, precip: {precip_val}, weather: {weather}")

In [ ]:
df_flights_weather_delay.columns